# GPT-OSS Weight Adjustment
Run Cell 0 first to clear disk, then proceed.

In [ ]:
# Cell 0: Clear disk space
!rm -rf ~/.cache/huggingface
!rm -rf /content/gpt-oss-20b
!rm -rf /content/hf_cache
!df -h

In [2]:
# Cell 1: Install (RESTART RUNTIME AFTER THIS)
!pip install --upgrade -qqq uv
try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
except: get_numpy = "numpy"
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers==4.56.2" \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
    git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers
print("\n*** RESTART RUNTIME NOW ***")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.3/23.3 MB 119.5 MB/s eta 0:00:0000:0100:01
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 15ms                                          
Audited 2 packages in 0.22ms

*** RESTART RUNTIME NOW ***


In [ ]:
# Cell 2: Check GPU + disk
!nvidia-smi
!df -h

In [ ]:
# Cell 3: Load model
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gpt-oss-20b",
    max_seq_length=512,
    load_in_4bit=True,
    offload_embedding=True,
)
print("Model loaded.")

In [ ]:
# Cell 4: Check GPU after load
!nvidia-smi

In [ ]:
# Cell 5: See weight names
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

In [ ]:
# Cell 6: Adjust weights (norms are not quantized)
import torch
layer = 0
with torch.no_grad():
    for name, param in model.named_parameters():
        if f"layers.{layer}" in name and "norm" in name:
            param.data *= 0.9
            print(f"Adjusted: {name}")
print("Done.")

Adjusted: model.layers.0.input_layernorm.weight
Adjusted: model.layers.0.post_attention_layernorm.weight
Done.


In [ ]:
# Cell 7: Test inference
prompt = "Hello, how are you?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0]))